In [ ]:
folderPath = "D:/"

In [ ]:
import numpy as np
import pandas as pd

def PrintGreen(text):
    print('\x1b[6;30;42m' + text + '\x1b[0m')
    
def PrintRed(text):
    print('\33[41m' + text + '\x1b[0m')

def LoadData(filename, rowsName, columnsName):
    newDataframe = pd.read_csv(filename, na_values = 'null')
    if newDataframe.shape[0] > 0 and newDataframe.shape[1] > 0:
        PrintGreen("Loading " + filename + " succeeded");
    else:
        PrintRed("Loading " + filename + " failed!");

    print(rowsName + " = " + str(newDataframe.shape[0]))
    print(columnsName + " = " + str(newDataframe.shape[1]))

    return newDataframe

# Ensure to show all columns
pd.set_option('display.max_columns', None)

In [ ]:
dataPosRotModelSpace = LoadData(folderPath + 'MotionMatchingDatabase_Poses_PosRot_ModelSpace_60Hz.csv', "Frames", "PoseComponents")
dataPosRotModelSpace.head(15)

In [ ]:
dataPosRotModelSpaceTesting = LoadData(folderPath + 'MotionMatchingDatabase_Poses_PosRot_ModelSpace_60Hz_Testing.csv', "Frames", "PoseComponents")
dataPosRotModelSpaceTesting.head(15)

In [ ]:
dataRotLocalSpace = LoadData(folderPath + 'MotionMatchingDatabase_Poses_Rot_LocalSpace_60Hz_Training.csv', "Frames", "PoseComponents")
dataRotLocalSpace.head(15)

In [ ]:
dataRotLocalSpaceTesting = LoadData(folderPath + 'MotionMatchingDatabase_Poses_Rot_LocalSpace_60Hz_Testing.csv', "Frames", "PoseComponents")
dataRotLocalSpaceTesting.head(15)

In [ ]:
dataFeatures = LoadData(folderPath + 'MotionMatchingDatabase_Features_60Hz_Training.csv', "Frames", "FeatureComponents")
dataFeatures.head(50)

In [ ]:
dataFeaturesTesting = LoadData(folderPath + 'MotionMatchingDatabase_Features_60Hz_Testing.csv', "Frames", "FeatureComponents")
dataFeaturesTesting.head(50)

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

posRotModelSpaceDataScalerTesting = preprocessing.StandardScaler()
posRotModelSpaceDataScalerTesting.fit(dataPosRotModelSpace.values)
dataPosRotModelSpaceScaledTesting = posRotModelSpaceDataScalerTesting.transform(dataPosRotModelSpace.values)

rotLocalSpaceDataScalerTesting = preprocessing.StandardScaler()
rotLocalSpaceDataScalerTesting.fit(dataRotLocalSpace.values)
dataRotLocalSpaceScaledTesting = rotLocalSpaceDataScalerTesting.transform(dataRotLocalSpace.values)

featureDataScalerTesting = preprocessing.StandardScaler()
featureDataScalerTesting.fit(dataFeatures.values)
dataFeaturesScaledTesting = featureDataScalerTesting.transform(dataFeatures.values)

featureDataScaler = preprocessing.StandardScaler()
dataFeaturesScaled = featureDataScaler.fit_transform(dataFeatures.values)
finalPoses = pd.DataFrame(dataPosRotModelSpaceScaledTesting, columns=dataPosRotModelSpace.columns.values)
finalPoses

In [ ]:
import onnxruntime
import torch

projector_session = onnxruntime.InferenceSession("D:/AutoEncoderModelSpace1layer1872to117SplitModelEncoder.onnx")
projector_input = dataPosRotModelSpaceScaledTesting
X = torch.tensor(projector_input, dtype=torch.float32)
projector_input = {"input": X.numpy()}
projector_outs = projector_session.run(None, projector_input)

In [ ]:
stepper_session = onnxruntime.InferenceSession("D:/Stepper2layer194to194RELUto194Epoch20LR1e-3Batch64NoBadFramesThreshold4.onnx")
stepper_input = np.concatenate((dataFeaturesScaledTesting, projector_outs[0]), axis=1) # concatenate horizontally
X = torch.tensor(stepper_input, dtype=torch.float32)
stepper_input = {"input": X.numpy()}
stepper_outs = stepper_session.run(None, stepper_input)

In [ ]:
stepper_input

In [ ]:
decompressor_session = onnxruntime.InferenceSession("D:/DecompressorLocalSpace2layer194to1248RELUto1248Epoch100LR1e-4Batch64.onnx")
decompressor_input = stepper_outs[0]
X = torch.tensor(decompressor_input, dtype=torch.float32)
decompressor_input = {"input": X.numpy()}
decompressor_outs = decompressor_session.run(None, decompressor_input)

In [ ]:
finalPoses = pd.DataFrame(decompressor_outs[0], columns=dataRotLocalSpace.columns.values)
finalPoses

In [ ]:
finalPoses.to_csv("D:/OG_Test.csv", index=False)